In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [ ]:
%run './imports.ipynb'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.6 MB/s eta 0:00:00


In [ ]:
%run './calc.ipynb'

In [ ]:
# import tensorflow as tf
# tf.config.experimental.set_visible_devices(tf.config.experimental.list_physical_devices('GPU'), 'GPU')

In [ ]:
def get_ids(folder_path):
    # Get all the files in the folder
    files = os.listdir(folder_path)
    # Extract the IDs from the filenames
    ids = []
    for file in files:
        if file.startswith("shhs1-") and file.endswith(".edf"):
            id_val = file[len("shhs1-"):-len(".edf")]
            try:
                id_val = int(id_val)
                ids.append(id_val)
            except ValueError:
                print(f"Warning: Invalid ID format in filename '{file}'. Skipping.")

    return ids

In [ ]:
def run_calc(df, splits):
    inputs=[]
    for idx in range(len(splits)):
        inputs.append([df,splits[idx]])
    pool=Pool(4)
    temp_= pool.map(calculations_cgc,inputs)

    output = temp_[0].append(temp_[1]).append(temp_[2]).append(temp_[3])
    return output

In [ ]:
def gc_table(df):
  df = pd.pivot_table(df, values='CGC_Value', index='X', columns='Y', aggfunc=lambda x: int(x > 0))
  df = df.fillna(0).astype(int)
  df = pd.DataFrame(df)
  df.index.name = None
  df.columns.name = None
  return df

In [ ]:
def runCLSTM(id):
  # with tf.device('/GPU:0'):
  df = pickle.load(open("./Data/Raw/MSAHBP_female/{}_data.p".format(id),"rb"))
  awake = df.loc[(df['sleep_stage'] == 0)]
  n12 = df.loc[(df['sleep_stage'] == 1) | (df['sleep_stage'] == 2)]
  n34 = df.loc[(df['sleep_stage'] == 3) | (df['sleep_stage'] == 4)]
  rem = df.loc[(df['sleep_stage'] == 5)]

  list_of_bands = pd.read_csv('./list_of_emg_lstm_signals.csv')
  splits=np.array_split(list_of_bands,4)
  inputs=[]
  # for idx in range(len(splits)):
  inputs.append([df,list_of_bands])
  df,input_bands = inputs[0]
  output_awake = run_calc(awake, splits)
  output_n12 = run_calc(n12, splits)
  output_rem = run_calc(rem, splits)
  output_n34 = run_calc(n34, splits)
  output_awake = pd.DataFrame(output_awake)
  output_n12 = pd.DataFrame(output_n12)
  output_rem = pd.DataFrame(output_rem)
  output_n34 = pd.DataFrame(output_n34)
  output_awake = output_awake[['X','Y','CGC_Value']]
  output_n12 = output_n12[['X','Y','CGC_Value']]
  output_rem = output_rem[['X','Y','CGC_Value']]
  output_n34 = output_n34[['X','Y','CGC_Value']]
  output_awake = gc_table(output_awake)
  output_n12 = gc_table(output_n12)
  output_rem = gc_table(output_rem)
  output_n34 = gc_table(output_n34)
  return output_awake, output_n12, output_n34, output_rem

In [ ]:
# ids = get_ids('./Data/Raw/Healthy_male')
# data = {
#     'Alpha': [0, 0, 0, 0, 0, 0, 0, 0],
#     'Beta': [0, 0, 0, 0, 0, 0, 0, 0],
#     'Delta': [0, 0, 0, 0, 0, 0, 0, 0],
#     'Gamma1': [0, 0, 0, 0, 0, 0, 0, 0],
#     'Gamma2': [0, 0, 0, 0, 0, 0, 0, 0],
#     'Sigma': [0, 0, 0, 0, 0, 0, 0, 0],
#     'Theta': [0, 0, 0, 0, 0, 0, 0, 0],
#     'hr': [0, 0, 0, 0, 0, 0, 0, 0]
# }

# o_awake = pd.DataFrame(data, index=['Alpha', 'Beta', 'Delta', 'Gamma1', 'Gamma2', 'Sigma', 'Theta', 'hr'])
# o_n12 = pd.DataFrame(data, index=['Alpha', 'Beta', 'Delta', 'Gamma1', 'Gamma2', 'Sigma', 'Theta', 'hr'])
# o_n34 = pd.DataFrame(data, index=['Alpha', 'Beta', 'Delta', 'Gamma1', 'Gamma2', 'Sigma', 'Theta', 'hr'])
# o_rem = pd.DataFrame(data, index=['Alpha', 'Beta', 'Delta', 'Gamma1', 'Gamma2', 'Sigma', 'Theta', 'hr'])
ids = [201446, 201449]
for id in ids:
  output_awake, output_n12, output_n34, output_rem = runCLSTM(id)
  with pd.ExcelWriter('./Outputs/LSTM_CGC/MSAHBP_female/{}-cgc.xlsx'.format(id)) as writer:
    output_awake.to_excel(writer, sheet_name = 'awake')
    output_n12.to_excel(writer, sheet_name = 'n12')
    output_n34.to_excel(writer, sheet_name = 'n34')
    output_rem.to_excel(writer, sheet_name = 'rem')
  # o_awake += output_awake
  # o_n12 += output_n12
  # o_n34 += output_n34
  # o_rem += output_rem